In [86]:
import pandas as pd
import nltk
nltk.download('punkt')

In [87]:
df = pd.read_csv("spotify_millsongdata.csv")

In [88]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [89]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [90]:
df.shape

(57650, 4)

In [91]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [92]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [93]:
df.head(10)

,artist,song,text
0,Van Halen,I'm The One,We came here to entertain you \r\nLeaving her...
1,Gordon Lightfoot,The First Time Ever I Saw Your Face,The first time ever I saw your face \r\nI tho...
2,Eddie Cochran,Drownin' All My Sorrows,"You ask me if I'm lonesome, you say you're fee..."
3,Nick Drake,Hazey Jane I,"Do you curse where you come from, \r\nDo you ..."
4,Nat King Cole,Looking Back,Looking ba-a-ack over my life \r\nI can see w...
5,Gino Vannelli,Love Me Now,Yes I know how much you love me \r\nbut can't...
6,LL Cool J,Stand By Your Man,Yeah this one goes out to all the ladies in th...
7,Ozzy Osbourne,Bark At The Moon,Screams break the silence \r\nWaking from the...
8,Elton John,Burning Buildings,It used to be a sweet sensation \r\nNo price ...
9,Air Supply,Faith,Easy come and easy go \r\nHard to get a hold ...


In [94]:
df['text'][0]

"We came here to entertain you  \r\nLeaving here we aggravate you  \r\nDon't you know it means the same to me, honey  \r\nI'm the one the one you love  \r\nCome on baby, show your love  \r\nHey, give it to me  \r\nI see a glow that fills this room  \r\nI see it rolling out of you  \r\nFeed her your message from above  \r\nI'm tellin' you, ow  \r\nShow, come on and show your love, ah, yeah  \r\n(Show your love)  \r\nOw, woo, oh, show (show) show your love babe, ah, yeah  \r\n(Show your...) Show it, ow  \r\nLook at all these little kids  \r\nTakin' care of the music biz  \r\nDon't their bus'ness take good care of me, honey  \r\nI'm the one the one you love  \r\nCome on baby, show your love  \r\nHey, give it to me  \r\nI see a glow that fills this room  \r\nI see it rolling out of you  \r\nFeed her your message from above  \r\nI'm telling you, ow  \r\nShow, show your love babe, ah, yeah  \r\n(Show your love)  \r\nOw, whoo, woo, show (show) show your love babe, ah, yeah  \r\n(Show your...)

In [95]:
# df = df.sample(5000)

In [96]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [97]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [98]:
df = df.assign(feedback=0)

In [99]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [100]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [102]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [103]:
similarity[0]

array([1.        , 0.0175531 , 0.00970593, ..., 0.01891506, 0.01219328,
       0.00896428])

In [104]:
df[df['song'] == df.iloc[0]['song']]

,artist,song,text,feedback
0,Van Halen,I'm The One,we came here to entertain you leav here we agg...,0


In [105]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [106]:
recommendation(df.iloc[0]['song'])

["Don't Be Cruel",
 'Wear Out The Turnpike',
 'Something Happened',
 'Rubberneck',
 'Heart Attack',
 'On The Road',
 'Move Over ( In Album Pearl )',
 "Cowgirl's Dream",
 'Enough Of The Night',
 'Boll Weevil Song',
 'Leave It At That',
 'Love Them Girls',
 'Calypso',
 'The 59th Street Bridge Song (Feelin Groovy)',
 'Red Light',
 'Love Song',
 'Ghouls Gone Wild',
 'All You Need Is Love',
 'The Glow Of Love',
 'Here He Comes (Lover Man)']

In [107]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [108]:
df.head()

,artist,song,text,feedback
0,Van Halen,I'm The One,we came here to entertain you leav here we agg...,0
1,Gordon Lightfoot,The First Time Ever I Saw Your Face,the first time ever i saw your face i thought ...,0
2,Eddie Cochran,Drownin' All My Sorrows,"you ask me if i 'm lonesom , you say you 're f...",0
3,Nick Drake,Hazey Jane I,"do you curs where you come from , do you swear...",0
4,Nat King Cole,Looking Back,look ba-a-ack over my life i can see where i c...,0
